### Include Adjacently modules

In [1]:
include("../src/graph.jl")
include("../src/io.jl")
include("../src/pr.jl")

PR

### Load Arxiv HEP-PH Directed Graph

See [Arxiv HEP-PH dataset](https://snap.stanford.edu/data/cit-HepPh.html)

In [2]:
@info("loading graph")

core = SimpleDiGraph(UInt32)

load_mgs3_graph(core, "../datasets/Arxiv_HEP-PH/Arxiv_HEP-PH_core.mgs")
#load_mgs4_graph(core, "../datasets/Arxiv_HEP-PH/Arxiv_HEP-PH_core.mgz")

# define constants
n = nv(core)
damping = .85
epsilon = 1e-8

# source and target node to be used in the graph
s = convert(UInt32,1)
t = convert(UInt32,100)

[ Info: loading graph


0x00000064

### Get "Reverse" Core

In [3]:
@info("getting rcore")
rcore = get_reverse_graph(core)

[ Info: getting rcore


{12711, 139981} directed simple UInt32 graph

### Compute Pagerank Vectors for Core and RCore

In [4]:
@info("computing Pagerank of core and rcore")

@time pr_core = PR(core, rcore, epsilon=epsilon)
#@time pr_rcore = PR(rcore, core, epsilon=epsilon)

[ Info: computing Pagerank of core and rcore
[ Info: computing Pagerank (size of graph 12711)


  0.221898 seconds (276.41 k allocations: 20.071 MiB, 66.54% compilation time)


12711-element Vector{Float64}:
 1.2180426083332923e-5
 5.576958680721473e-5
 0.0001523018183933433
 1.881325654228591e-5
 2.6916039325516338e-5
 3.021143080036028e-5
 0.002712795282615076
 1.7131225142005417e-5
 0.002050165700823033
 4.4210978352731306e-5
 0.0014305859177234886
 0.0008125784974850691
 0.00014323071476047894
 ⋮
 2.41192412710315e-5
 1.863458748135917e-5
 1.2181834342831553e-5
 1.29865309587917e-5
 1.4277488989680935e-5
 1.3850475578385906e-5
 1.2342431035639167e-5
 1.4356399707917643e-5
 1.211041703709851e-5
 1.3283364288518863e-5
 1.3274575973293074e-5
 1.598313331166969e-5

### Compute Personalized Pagerank Vectors (for nodes 1 and 100)

In [5]:
@info("computing personalized Pageranks for nodes 1 and 100")
@time pr_s = PPR(s, core, rcore, epsilon=epsilon)
#@time pr_t = PPR(t, rcore, core, epsilon=epsilon)

[ Info: computing personalized Pageranks for nodes 1 and 100
[ Info: computing personalized Pagerank (size of graph 12711, source 1)


  0.208636 seconds (199.49 k allocations: 16.261 MiB, 60.49% compilation time)


12711-element Vector{Float64}:
 0.15000003731617217
 0.03886675940776586
 0.050230378864137455
 0.025500006346721265
 0.029835053246013862
 0.025500045509412397
 0.017128009924146623
 0.007225001798238081
 0.04550645808618301
 0.013936024708081415
 0.01852803880126476
 0.011869258242420914
 0.01101338748178447
 ⋮
 9.463316603290713e-23
 2.681298118225403e-23
 2.0685033052419324e-22
 6.608469564754296e-24
 7.75077605921725e-21
 5.388743109387597e-10
 4.420936212067362e-8
 6.450921883670794e-19
 3.603377132829556e-24
 2.190412794090886e-23
 2.3470375731396953e-24
 8.29226903090909e-8

In [6]:
# compute Monte Carlo PR
niter = 100
@info("compute Pagerank (Monte Carlo)")
@time pr_mc = PR(core, niter)

@info("pr_core <-> pr_mc: ", chebyshev(pr_core, pr_mc))

[ Info: compute Pagerank (Monte Carlo)
[ Info: computing Monte-Carlo Pagerank (size of graph 12711)


 26.435971 seconds (5.83 M allocations: 180.772 GiB, 13.50% gc time, 1.35% compilation time)


┌ Info: pr_core <-> pr_mc: 
└   chebyshev(pr_core, pr_mc) = 0.00011162320816283933


In [7]:
# P = D^-1 * A matrix
matrix computation
@info("getting P matrix")
@time P = get_sparse_P_matrix(core)

[ Info: getting P matrix


  1.345065 seconds (2.54 M allocations: 148.511 MiB, 3.72% gc time, 99.17% compilation time)


12711×12711 SparseMatrixCSC{Float64, UInt32} with 139981 stored entries:
⣿⣾⡿⣷⣿⣿⣿⣿⣿⣷⣾⣷⣿⣶⣾⣿⡮⣽⣿⣿⣿⣿⣷⣷⣿⣷⣯⣷⢶⣾⣬⣽⣾⡧⢾⣾⣛⣌⣇⣂
⣿⣿⣿⣿⣿⣏⣯⣿⡭⣿⣿⣿⣯⣯⣿⣻⣇⣸⣿⣿⣯⣻⣟⣻⣏⣿⢮⣯⡇⣼⢎⡫⢭⢺⡥⠩⠫⠉⣔⣀
⣿⣿⣿⣿⣿⣿⣿⣿⣻⣿⣿⣿⣷⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣕⣿⣿⣶⣿⣦⡿⣿⢻⡿⣪⣷⠤
⣽⣿⣇⣭⣿⣿⣿⣯⣿⣿⣿⡿⣿⡿⢻⣿⣿⣿⣽⣽⣿⣾⣿⡟⢿⣯⣵⣯⣧⢿⣫⣾⣽⣴⣧⣬⡅⣅⢤⢀
⢿⣿⣧⣭⣷⣿⣿⣿⣿⣿⣿⣶⣿⣿⣽⣿⣿⢿⣿⣯⣽⣯⣯⡫⣼⢿⠿⢧⣿⣫⢜⡿⣛⣚⡏⡇⡖⢫⠞⠆
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣟⣿⣿⣿⣿⣿⣾⣝⣿⣿⣧⣯⡗⣻⣓⣷⣸⣾⣷⣾⣭⣠⣵⠵
⣻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⣿⣿⣿⣿⣿⡟⣟⣧⣺⣿⣿⣿⣷⡧⣾⢼⣶⠴
⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⣿⣿⣿⣿⣾⣷⣿⣿⣿⣿⣿⣿⣟⣿⣿⣿⣿⣿⣿⡯⣽⢾⣿⣿⢭⣷⢮⣗⠏
⣿⣿⣏⣻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣝⣿⣿⣿⣻⣻⣿⣿⣯⣻⣿⣿⣿⣿⣯⣿⣫⣻⡿⣷⣿⣿⣻⣷⣲⡾
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧⣿⣿⣟⣿⣽⣿⣻⡿⡻⣮⠅
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣷⣿⣿⣿⣿⣿⢿⣿⣿⣿⣿⣿⣾⣿⣿⣿⠽
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣽⣽⣿⣯⣿⣽⣽⡮
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡟⣽⣿⣿⣯⣽⣿⣿⣿⡤
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡯⣿⣟⢿⣷⣷⣿⣿⣷⣟⡲⣟
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⢗
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣟⣿⣿⣿⣿⣻⣯⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣧
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿
⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⢿⣿⣿⣿⣿⣿⣿⣿⣿⡿⣿⣿⡿⣿⣿⣿⣿⣿⣿⢿⢿⢻⣿⣽⣿⣿⣿

In [8]:
@info("compute Pagerank (power iteration)")
ppr = zeros(Float64,n)
ppr[s] = 1.
@time pr_pi = PR(P, epsilon=epsilon)

@info("computing personalized Pageranks for node 1 (power iteration)")
@time pr_pi_s = PR(P, ppr=ppr, epsilon=epsilon)

@info("pr_core <-> pr_pi: ", chebyshev(pr_core, pr_pi))
@info("pr_s <-> pr_pi_s: ", chebyshev(pr_s, pr_pi_s))

[ Info: compute Pagerank (power iteration)


  0.379660 seconds (615.84 k allocations: 57.341 MiB, 5.48% gc time, 95.42% compilation time)
  0.065851 seconds (12.13 k allocations: 29.406 MiB, 21.92% gc time, 38.52% compilation time)


[ Info: computing personalized Pageranks for node 1 (power iteration)
┌ Info: pr_core <-> pr_pi: 
└   chebyshev(pr_core, pr_pi) = 2.7755575615628914e-17
┌ Info: pr_s <-> pr_pi_s: 
└   chebyshev(pr_s, pr_pi_s) = 5.3649326516025386e-8


In [9]:
@info("compute non-linear Pagerank (power iteration)")
ppr = zeros(Float64, n)
ppr[s] = 1.
# change damping
damping = 0.95
max_iter = 15

@time pr_pi_nl = PR(P, tanh, damping=damping, epsilon=epsilon, max_iter=max_iter)

@info("pr_pi <-> pr_pi_nl: ", chebyshev(pr_pi, pr_pi_nl))


[ Info: compute non-linear Pagerank (power iteration)


  0.279461 seconds (732.08 k allocations: 49.556 MiB, 6.11% gc time, 97.13% compilation time)


┌ Info: pr_pi <-> pr_pi_nl: 
└   chebyshev(pr_pi, pr_pi_nl) = 0.06966279339435628


In [13]:
using GraphPlot

layout=(args...)->spring_layout(args...; C=20)
gplot(core, layout=layout)